<h1 style="text-align: center;">Mesh Generation</h1>

This Notebook is devoted to the generation of mesh in *femtoscope*. It mainly relies on two Python packages:
- gmsh (4.11.1)
- meshio (4.4.6)

We will see how to generate meshes from scratch using the Gmsh Python API (which is similar to what is done in SELCIE) as well as the generation of meshes from pre-written .geo files (Gmsh's built-in scripting language).

**Warning**: be sure to install the above-mentioned version of `meshio` as newer versions might cause errors.

**If you have questions/comments or want to report a bug, please send me an email at <a href="mailto:hugo.levy@onera.fr">hugo.levy@onera.fr</a>**

## Physical groups handling in *femtoscope*

*femtoscope* builds on top on the FEM package Sfepy, which is compatible with VTK *(Visualization Toolkit legacy file format)* mesh files. Gmsh allows the definition of *physical groups*, which is a convenient way of grouping model entities. Meshes in *femtoscope* must comply with the following numbering rules:
- vertex entities must be numbered between 0 and 99
- edge entities must be numbered between 100 and 199
- facet entities must be numbered between 200 and 299
- cell entities must be numbered from 300

Please refer to https://sfepy.org/doc-devel/users_guide.html#regions for the definition of *vertex*, *edge*, *facet* and *cell*.

In [1]:
# Add femtoscope to the path
%reset
import sys
sys.path.append("../") # go to parent dir

## Mesh generation from scratch

### 1st example: meshing a rectangle

The default numbering of entities in this case will be as follows
<img src="../images/rectangle_numbering.png" alt="isolated" width="400"/>
Suppose we will not make use of edges 200, 202 and 203. We can ignore these edges (or any other physical entity) with the argument `ignoreTags` passed to the `generate_mesh` method.


In [2]:
from femtoscope.inout.meshfactory import MeshingTools

meshfile = 'rectangle.vtk'
MT = MeshingTools(dimension=2)
MT.create_rectangle(xll=0, yll=-1, dx=5, dy=2, centered=False)
MT.create_subdomain(CellSizeMin=0.5, CellSizeMax=0.5, DistMax=0.0)
MT.generate_mesh(meshfile, show_mesh=True, unique_boundary=False,
                 ignoreTags=[200, 202, 203])

sfepy: reading mesh (D:\hlevy\Documents\microscope2\femtoscope\script\..\data\mesh\rectangle.vtk)...
sfepy:   number of vertices: 68
sfepy:   number of cells:
sfepy:     1_2: 28
sfepy:     2_3: 106
sfepy: ...done in 0.00 s


'D:\\hlevy\\Documents\\microscope2\\femtoscope\\script\\..\\data\\mesh\\rectangle.vtk'

The mesh is saved at the path printed just above.
Rectangular meshes are particularly useful when working in polar coordinates $(r, \theta)$ instead of the traditional Cartesian coordinates $(x, y)$. Indeed, a disk in Cartesian coordinates is mapped to a rectangle in polar coordinates. While a circle cannot be perfectly approximated by a finite set of straight line, a rectangle can!

### 2nd example: Multiple cell groups

In [3]:
from femtoscope.inout.meshfactory import MeshingTools

symmetry = False
Ngamma = 200
Rcut = 5.0
sa = 1.0
sc = 0.5

meshfile = 'disk.vtk'
MT = MeshingTools(dimension=2)
s1 = MT.create_ellipse(sa, sc, xc=0, yc=0)
MT.create_subdomain(CellSizeMin=0.05, CellSizeMax=0.2, DistMin=sa/5, DistMax=3*sa)
s2 = MT.create_disk_from_pts(Rcut, N=Ngamma) # impose vertices on gamma
s12 = MT.subtract_shapes(s2, s1, removeObject=True, removeTool=False)
MT.create_subdomain(CellSizeMin=0.1, CellSizeMax=0.3, DistMax=0.0)
MT.generate_mesh(meshfile, show_mesh=True, ignoreTags=[200], symmetry=symmetry, embed_center=False)

sfepy: reading mesh (D:\hlevy\Documents\microscope2\femtoscope\script\..\data\mesh\disk.vtk)...
sfepy:   number of vertices: 6183
sfepy:   number of cells:
sfepy:     1_2: 296
sfepy:     2_3: 12165
sfepy: ...done in 0.03 s


'D:\\hlevy\\Documents\\microscope2\\femtoscope\\script\\..\\data\\mesh\\disk.vtk'

Here, we made use of geometry/topology functions such as `create_ellipse` to create the central ellipse, or `subsract_shapes` ti substract two topological entities.

## Mesh generation from .geo files

Another way to generate a mesh suitable for *femtoscope* FEM computations is to start from an existing Gmsh geometry file. Such files have the .geo extension and some examples can be found in the directory `data\mesh\geo`.

In the example down below, we pass the argument `param_dic` to the function `mesh_from_geo` in order to directly modify some numerical values of the corresponding .geo file.

In [4]:
from femtoscope.inout.meshfactory import mesh_from_geo

Rcut = 5.0
sa = 1.0
sc = 0.5
mesh_from_geo('mesh_theta.geo', show_mesh=True, param_dic={'size' : 0.1, 'Rc' : Rcut, 'sa' : sa})

sfepy: reading mesh (D:\hlevy\Documents\microscope2\femtoscope\script\..\data\mesh\mesh_theta.vtk)...
sfepy:   number of vertices: 1972
sfepy:   number of cells:
sfepy:     1_2: 32
sfepy:     2_3: 3778
sfepy: ...done in 0.02 s


'D:\\hlevy\\Documents\\microscope2\\femtoscope\\script\\..\\data\\mesh\\mesh_theta.vtk'

## The specific case of 1D meshes

Contrary to 2D and 3D meshes, 1D meshes are not generated using Gmsh. Instead, we rely on some Sfepy built-in functions to achieve this.

In [5]:
from sfepy.examples.dg.example_dg_common import get_gen_1D_mesh_hook
X1 = 0.0
XN = Rcut
n_nod = int(100*Rcut) + 1
n_el = n_nod - 1
mesh1D = get_gen_1D_mesh_hook(X1, XN, n_nod).read(None)

The mesh is not saved in `data\mesh\` but is nonetheless accessible through the variable `mesh1D`.